<h1><center>TP4 - Ex.1</center></h1>
<p><center>Abril 25, 2024</center></p>


### Estruturas Criptográficas

PG53886, Ivo Miguel Alves Ribeiro

A95323, Henrique Ribeiro Fernandes

**1. Implemente um protótipo do esquema descrito no “draft” FIPS 204 que deriva do algoritmo Dilithium.**

In [1]:
def bitrev7(n):
	return int(f"{n:07b}"[::-1], 2)

def NTT(f_in, Q):
	f_out = f_in.copy()
	k = 1
	for log2len in range(7, 0, -1):
		length = 2**log2len
		for start in range(0, 256, 2 * length):
			zeta = pow(17, bitrev7(k), Q)
			k += 1
			for j in range(start, start + length):
				t = (zeta * f_out[j + length]) % Q
				f_out[j + length] = (f_out[j] - t) % Q
				f_out[j] = (f_out[j] + t) % Q
	return f_out

def NTT_inv(f_in, Q):
	f_out = f_in.copy()
	k = 127
	for log2len in range(1, 8):
		length = 2**log2len
		for start in range(0, 256, 2 * length):
			zeta = pow(17, bitrev7(k), Q)
			k -= 1
			for j in range(start, start + length):
				t = f_out[j]
				f_out[j] = (t + f_out[j + length]) % Q
				f_out[j + length] = (zeta * (f_out[j + length] - t)) % Q

	for i in range(256):
		f_out[i] = (f_out[i] * 3303) % Q 

	return f_out

def NTT_mult(a, b, Q):
	c = []
	for i in range(128):
		a0, a1 = a[2 * i: 2 * i + 2]
		b0, b1 = b[2 * i: 2 * i + 2]
		c.append((a0 * b0 + a1 * b1 * pow(17, 2*bitrev7(i)+1, Q)) % Q)
		c.append((a0 * b1 + a1 * b0) % Q)
	return c

def polyADD(a, b, Q):
	return [(x + y) % Q for x, y in zip(a, b)]

def polySUB(a, b, Q):
	return [(x - y) % Q for x, y in zip(a, b)]

def polyMULT(a, b, Q):
	return [(x * y) % Q for x, y in zip(a, b)]

In [2]:
import os
from pmain.dilithium import Dilithium2, Dilithium3, Dilithium5
from sage.all import PolynomialRing, GF, QuotientRing, Integer
from pmain.shake_wrapper import Shake128, Shake256
from pmain.utils import *
from pmain.modules import *


class Dilithium:
    def __init__(self, type):
        if type == "dilithium2":
            self.n = 256
            self.q = 8380417
            self.d = 13
            self.k = 4
            self.l = 4
            self.eta = 2
            self.eta_bound = 15
            self.tau = 39
            self.omega = 80
            self.gamma_1 = 2**17
        elif type == "dilithium3":
            self.n = 256
            self.q = 8380417
            self.d = 13
            self.k = 6
            self.l = 5
            self.eta = 4
            self.eta_bound = 9
            self.tau = 49
            self.omega = 55
            self.gamma_1 = 2**19
        elif type == "dilithium5":
            self.n = 256
            self.q = 8380417
            self.d = 13
            self.k = 8
            self.l = 7
            self.eta = 2
            self.eta_bound = 15
            self.tau = 60
            self.omega = 75
            self.gamma_1 = 2**19
        self.gamma_2 = 95232
        self.beta = self.tau * self.eta
        R = PolynomialRing(GF(self.q), 'x', implementation='NTL')
        x = R.gen()
        f = x**self.n + 1
        self.R = QuotientRing(R, R.ideal(f))
        self.M = Module(self.R)
        self.drbg = None
        self.random_bytes = os.urandom

    
    def keygen(self):
        # Random seed
        zeta = self.random_bytes(32)
        # Expand with an XOF (SHAKE256)
        seed_bytes = self._h(zeta, 128)
        # Split bytes em partes para usar 
        rho, rho_prime, K = seed_bytes[:32], seed_bytes[32:96], seed_bytes[96:]
        # Generate matrix A ∈ R^(kxl)
        A = self._expandA(rho)
        # Generate the error vectors s1 ∈ R^l, s2 ∈ R^k
        s1, s2 = self._expandS(rho_prime)   
        #representação NTT de s1    
        s1_hat = [NTT(s1[i], self.q) for i in range(self.l)]
        # Matrix multiplication
        t = []
        for i in range(self.k):
            sum = s2[i]
            for j in range(self.k):
                sum = polyADD(NTT_mult(A[j][i], s1_hat[j], self.q), sum, self.q)
            t.append(sum)
        # Decompor t em t1 e t0
        t1, t0 = self.power_2_round(t)
        # Pack up the bytes para a PK
        pk = self._pack_pk(rho, t1)
        tr = self._h(pk, 32)  
        # Pack bytes para a Sk
        sk = self._pack_sk(rho, K, tr, s1, s2, t0)
        return pk, sk
    
    def sign(self, sk_bytes, m):
        # Desempacotar the secret key
        rho, K, tr, s1, s2, t0 = self._unpack_sk(sk_bytes)
        # Generate matrix A ∈ R^(kxl)
        A = self._expandA(rho)
        # Set seeds and nonce (kappa)
        mu = self._h(tr + m, 64)
        kappa = 0
        rho_prime = self._h(K + mu, 64)
        # Precompute NTT representation
        s1_hat = [NTT(s1[i], self.q) for i in range(self.l)]
        s2_hat = [NTT(s2[i], self.q) for i in range(self.k)]
        t0_hat = [NTT(t0[i], self.q) for i in range(self.k)]
        alpha = self.gamma_2 << 1
        # Loop de tentativa de assinatura valida
        while True:
            # Expandir o rho com a nounce
            y = self._expandMask(rho_prime, kappa)
            # Compute NTT
            y_hat = [NTT(y[i], self.q) for i in range(self.k)]
            
            # increment the nonce
            kappa += self.l
            
            # Multiplication Matrix and y_hat
            w = []
            for i in range(self.k):
                sum = [0]*self.n
                for j in range(self.k):
                    # transposta de A_hat por trocar o i com o j
                    sum = polyADD(NTT_inv(NTT_mult(A[i][j], y_hat[j], self.q), self.q),sum, self.q)
                w.append(sum)

            # Extract out both the high and low bits
            w1, w0 = self.decompose(w, alpha)
            
            # Create challenge polynomial
            w1_bytes = self.bit_pack_w(w1, self.gamma_2)
            c_tilde = self._h(mu + w1_bytes, 32)
            c = self._sample_in_ball(c_tilde)
            
            # Store c in NTT form
            c_hat = NTT(c, self.q)
            
            # Calculate z = c_hat * s1_hat + y
            z = []
            for i in range(self.k):
                sum = y[i]
                sum = polyADD(NTT_inv(polyMULT(c_hat, s1_hat[i], self.q), self.q),sum, self.q)
                z.append(sum)

            # Calculate w0_minus_cs2 = c_hat * s2_hat - w0
            w0_minus_cs2 = []
            for i in range(self.k):
                sum = w0[i]
                sum = polySUB(NTT_inv(polyMULT(c_hat, s2_hat[i], self.q), self.q),sum, self.q)
                w0_minus_cs2.append(sum)
            
            # Extract out both the high and low bits
            r1, r0 = self.decompose(w0_minus_cs2, alpha)

            # Check limit bounds 
            if self.check_norm_bound(z, self.gamma_1 - self.beta):
                continue

            # Calculate c_t0 = t0 * c_hat + y
            c_t0 = []
            for i in range(self.k):
                sum = y[i]
                sum = polyADD(NTT_inv(NTT_mult(c_hat, t0_hat[i], self.q), self.q),sum, self.q)
                c_t0.append(sum)
            
            # Check limit bounds 
            if self.check_norm_bound(c_t0, self.gamma_2):
                continue
            
            w0_minus_cs2_plus_ct0 = w0_minus_cs2 + c_t0
            h = self._make_hint(w0_minus_cs2_plus_ct0, w1, alpha)   
            # Check limit bounds          
            if self._sum_hint(h) > self.omega:
                continue
            
            # Return packed bytes as sign
            return self._pack_sig(c_tilde, z, h)
        
    def verify(self, pk_bytes, m, sig_bytes):
        # Unpack pk
        rho, t1 = self._unpack_pk(pk_bytes)
        # Unpack sign 
        c_tilde, z, h = self._unpack_sig(sig_bytes)
        
        # Check limit bounds
        if self._sum_hint(h) > self.omega:
            return False
            
        if z.check_norm_bound(self.gamma_1 - self.beta):
            return False
        # Generate matrix A ∈ R^(kxl)
        A = self._expandA(rho)
        
        # Calculate mu and c
        tr = self._h(pk_bytes, 32)
        mu = self._h(tr + m, 64)
        c = self._sample_in_ball(c_tilde)
        
        # Convert to NTT for computation
        NTT(c, self.q)
        z_hat = [NTT(z[i], self.q) for i in range(self.l)]
        
        t1 = t1.scale(1 << self.d)
        t1_hat = [NTT(t1[i], self.q) for i in range(self.l)]
        
        Az_minus_ct1 = (A @ z_hat) - t1_hat
        Az_minus_ct1.from_ntt()
        # Multiplication Matrix and z_hat
        w = []
        for i in range(self.k):
            sum = [0]*self.n
            for j in range(self.k):
                # transposta de A_hat por trocar o i com o j
                sum = polyADD(NTT_inv(NTT_mult(A[i][j], z_hat[j], self.q), self.q),sum, self.q)
            w.append(sum)
        # Subtract t1_hat
        Az_minus_ct1 = polySUB(w, t1_hat, self.q)
        
        # Ajust w_prime with h
        w_prime = self._use_hint(h, Az_minus_ct1, 2*self.gamma_2)
        w_prime_bytes = w_prime.bit_pack_w(self.gamma_2)
        
        # Verify sign 
        return c_tilde == self._h(mu + w_prime_bytes, 32)
    
    def set_drbg_seed(self, seed):
        #self.drbg = AES256_CTR_DRBG(seed)
        self.random_bytes = self.drbg.random_bytes
        
    def reseed_drbg(self, seed):
        if self.drbg is None:
            raise Warning(f"Cannot reseed DRBG without first initialising. Try using `set_drbg_seed`")
        else:
            self.drbg.reseed(seed)
             
    def _h(self, input_bytes, length):
        return Shake256.digest(input_bytes, length)
             
    def _make_hint(self, v1, v2, alpha):
        matrix = [[self._make_hint_poly(p1, p2, alpha) for p1, p2 in zip(v1.rows[i], v2.rows[i])]
                   for i in range(v1.m)] 
        return self.M(matrix)
        
    def _use_hint(self, v1, v2, alpha):
        matrix = [[self._use_hint_poly(p1, p2, alpha) for p1, p2 in zip(v1.rows[i], v2.rows[i])]
                  for i in range(v1.m)]
        return self.M(matrix)
    
    def _make_hint_poly(self, p1, p2, alpha):
        coeffs = [make_hint(r, z, alpha, self.q) for r, z in zip(p1.coeffs, p2.coeffs)]
        return self.R(coeffs)
        
    def _use_hint_poly(self, p1, p2, alpha):
        coeffs = [use_hint(h, r, alpha, self.q) for h, r in zip(p1.coeffs, p2.coeffs)]
        return self.R(coeffs)

    def _sum_hint(self, hint):
        return sum(c for row in hint.rows for p in row for c in p)

    def _sample_in_ball(self, seed):     
        def rejection_sample(i, xof):
            while True:
                j = xof.read(1)
                j = int.from_bytes(j, "little")
                if j <= i: 
                    return j
        
        # Initialise the XOF
        Shake256.absorb(seed)
        
        # Set the first 8 bytes for the sign, and leave the rest for
        # sampling.
        sign_bytes = Shake256.read(8)
        sign_int = int.from_bytes(sign_bytes, "little")
        
        # Set the list of coeffs to be 0
        coeffs = [0 for _ in range(self.n)]
        # Now set tau values of coeffs to be ±1
        for i in range(256 - self.tau, self.n):
            j = rejection_sample(i, Shake256)
            coeffs[i] = coeffs[j]
            coeffs[j] = 1 - 2*(sign_int & 1)
            sign_int >>= 1
            
        return self.R(coeffs).list()
        
    def _sample_error_polynomial(self, rho_prime, i):
        def rejection_sample(xof):
            while True:
                js = []
                # Consider two values for each byte (top and bottom four bits)
                j  = xof.read(1)
                j  = int.from_bytes(j, "little")
                j0 = j & 0x0F
                j1 = j >> 4
                # rejection sample
                if j0 < self.eta_bound:
                    if self.eta == 2: j0 %= 5
                    js.append(self.eta - j0)
                if j1 < self.eta_bound:
                    if self.eta == 2: j1 %= 5
                    js.append(self.eta - j1)
                if js:
                    return js
        # Initialise the XOF
        seed = rho_prime + int.to_bytes(i, 2, "little")
        Shake256.absorb(seed)
    
        # Sample bytes for all n coeffs
        coeffs = []
        while len(coeffs) < self.n:
            js = rejection_sample(Shake256)
            coeffs += js

        # Remove the last byte if we ended up overfilling
        if len(coeffs) > self.n:
            coeffs = coeffs[:self.n]
        
        return self.R(coeffs).list()
    
    def _sample_matrix_polynomial(self, rho, i, j):
        def rejection_sample(xof):
            while True:                
                j_bytes = xof.read(3)
                j = int.from_bytes(j_bytes, "little")
                j &= 0x7FFFFF
                if j < self.q:
                    return j

        # Initialise the XOF
        seed = rho + bytes([j, i])
        Shake128.absorb(seed)
        coeffs = [rejection_sample(Shake128) for _ in range(self.n)]
        return self.R(coeffs).list()
    
    def _sample_mask_polynomial(self, rho_prime, i, kappa):                            
        if self.gamma_1 == (1 << 17):
            bit_count = 18
            total_bytes = 576 # (256 * 18) / 8
        else:
            bit_count = 20
            total_bytes = 640 # (256 * 20) / 8
        
        # Initialise the XOF
        seed = rho_prime + int(Integer(kappa + i)).to_bytes(2, "little")
        xof_bytes = Shake256.digest(seed, total_bytes)
        r = int.from_bytes(xof_bytes, 'little')
        mask = (1 << bit_count) - 1
        coeffs = [self.gamma_1 - ((r >> bit_count*i) & mask) for i in range(self.n)]
        
        return coeffs
        
    def _expandA(self, rho):
        matrix = [[self._sample_matrix_polynomial(rho, i, j)
                   for j in range(self.l)]
                   for i in range(self.k)]
        return matrix
        
    def _expandS(self, rho_prime):
        s1 = [self._sample_error_polynomial(rho_prime, i) 
                       for i in range(self.l)]
        s2 = [self._sample_error_polynomial(rho_prime, i) 
                       for i in range(self.l, self.l+self.k)]
        return s1, s2
        
    def _expandMask(self, rho_prime, kappa):
        elements = [self._sample_mask_polynomial(rho_prime, i, kappa)
                    for i in range(self.l)]
        return elements
    
    def bit_pack(self, coeffs, n_bits, n_bytes):
        r = 0
        for c in coeffs:
            r <<= n_bits
            r |= Integer(c)
        return int(r).to_bytes(n_bytes, 'little')
    
    def _pack_pk(self, rho, t1):
        pk = rho
        for i in range(self.k):
            pk += self.bit_pack(t1[i], 10, 320)
        return pk
    
    def bit_pack_s(self, s, eta):
        altered_coeffs = [eta - c for c in s]
        # Level 2 and 5 parameter set
        if eta == 2:
            return self.bit_pack(altered_coeffs, 3, 96)
        # Level 3 parameter set
        elif eta == 4:
            return self.bit_pack(altered_coeffs, 4, 128)
        else:
            raise ValueError("Expected eta to be either 2 or 4")

    def _pack_sk(self, rho, K, tr, s1, s2, t0):
        sk = rho + K + tr
        for i in range(self.l):
            sk += self.bit_pack_s(s1[i], self.eta)
        for i in range(self.k):  
            sk += self.bit_pack_s(s2[i], self.eta)
        print(len(sk))
        for i in range(self.k): 
            sk += self.bit_pack(t0[i], 13, 416)
        return sk
    
    def bit_pack_w(self, coeffs, gamma_2):
        pack = b''
        # Level 2 parameter set
        if gamma_2 == 95232:
            for i in range(self.k): 
                pack += self.bit_pack(coeffs[i], 6, 192)
        # Level 3 and 5 parameter set
        elif gamma_2 == 261888:
            for i in range(self.k): 
                pack += self.bit_pack(coeffs[i], 4, 128)
        else:
            raise ValueError("Expected gamma_2 to be either (q-1)/88 or (q-1)/32")
        return pack
    
    def bit_pack_z(self, coeffs, gamma_1):
        pack = b''
        # Level 2 parameter set
        if gamma_1 == (1 << 17):
            for poly in coeffs:
                altered_coeffs = [gamma_1 - c for c in poly]
                pack += self.bit_pack(altered_coeffs, 18, 576)
        # Level 3 and 5 parameter set
        elif gamma_1 == (1 << 19):
            for poly in coeffs:
                altered_coeffs = [gamma_1 - c for c in poly]
                pack += self.bit_pack(altered_coeffs, 20, 640)
        else:
            raise ValueError("Expected gamma_1 to be either 2^17 or 2^19")
        return pack
        
    def _pack_h(self, h):
        non_zero_positions = [[i for i,c in enumerate(poly.coeffs) if c == 1]
                               for row in h.rows for poly in row]
        packed  = []
        offsets = []
        for positions in non_zero_positions:
            packed.extend(positions)
            offsets.append(len(packed))

        padding_len = (self.omega - offsets[-1])
        packed.extend([0 for _ in range(padding_len)])
        return bytes(packed + offsets)
        
    def _pack_sig(self, c_tilde, z, h):
        print(z)
        return c_tilde + self.bit_pack_z(z, self.gamma_1) + self._pack_h(h)
        
    def _unpack_pk(self, pk_bytes):
        rho, t1_bytes = pk_bytes[:32],  pk_bytes[32:]
        t1 = self.M.bit_unpack_t1(t1_bytes, self.k, 1)
        return rho, t1
    
    def bit_unpack(self, input_bytes, n_bits):
        if len(input_bytes) % 8*n_bits != 0:
            raise ValueError("Input bytes do not have a length compatible with the bit length")
        r = int.from_bytes(input_bytes, 'little')
        mask = (1 << n_bits) - 1
        coeffs = []
        for _ in range(self.n):
            # Extrai o coeficiente aplicando a máscara
            coeff = r & mask
            coeffs.append(coeff)
            # Desloca os bits para a direita para o próximo coeficiente
            r >>= n_bits
        return list(reversed(coeffs))
    
    def bit_unpack_s(self, input_bytes):
        # Level 2 and 5 parameter set
        if self.eta == 2:
            altered_coeffs = self.bit_unpack(input_bytes, 3)
        # Level 3 parameter set
        elif self.eta == 4:
            altered_coeffs = self.bit_unpack(input_bytes, 4)
        else:
            raise ValueError("Expected eta to be either 2 or 4")
        return [(self.eta - c)%self.q for c in altered_coeffs] 
    
    def _unpack_sk(self, sk_bytes):
        if self.eta == 2:
            s_bytes = 96
        else:
            s_bytes = 128
        s1_len = s_bytes * self.l
        s2_len = s_bytes * self.k
        t0_len = 416 * self.k
        if len(sk_bytes) != 3*32 + s1_len + s2_len + t0_len:
            raise ValueError("SK packed bytes is of the wrong length")
        rho, K, tr = sk_bytes[:32], sk_bytes[32:64], sk_bytes[64:96]
        
        # Unpack vector bytes
        s1 = []
        for i in range(self.l):
            s1.append(self.bit_unpack_s(sk_bytes[96+s_bytes*i:96+s_bytes+s_bytes*i]))
        l = 96 + s1_len
        s2 = []
        for i in range(self.k):
            s2.append(self.bit_unpack_s(sk_bytes[l+s_bytes*i:l+s_bytes+s_bytes*i]))
        l += s2_len
        t0 = []
        for i in range(self.k):
            t0.append(self.bit_unpack(sk_bytes[l+416*i:l+416*(i+1)], 13))
        return rho, K, tr, s1, s2, t0
    
    def _unpack_h(self, h_bytes):
        offsets = [0] + list(h_bytes[-self.k:])
        non_zero_positions = [list(h_bytes[offsets[i]:offsets[i+1]]) for i in range(self.k)]
        
        matrix = []
        for poly_non_zero in non_zero_positions:
            coeffs = [0 for _ in range(self.n)]
            for non_zero in poly_non_zero:
                coeffs[non_zero] = 1
            matrix.append([self.R(coeffs)])
        return self.M(matrix)
        
    def _unpack_sig(self, sig_bytes):
        c_tilde = sig_bytes[:32]
        z_bytes = sig_bytes[32: -(self.k + self.omega)]
        h_bytes = sig_bytes[-(self.k + self.omega):]
        
        z = self.M.bit_unpack_z(z_bytes, self.l, 1, self.gamma_1)
        h = self._unpack_h(h_bytes)
        return c_tilde, z, h
    
    def power_2_round(self, poly):
        t0 = []
        t1 = []
        for i in range(self.k):
            r1_ = []
            r0_ = []
            for c in poly[i]:
                r_plus = Integer(c % self.q)  # Certificando que r_plus é um inteiro do Sage
                r0 = Integer(r_plus % (2^self.d))  # Usando inteiros do Sage para operações de módulo
                r1 = Integer((r_plus - r0) // (2^self.d)) 
                r1_.append(r1)
                r0_.append(r0)
            t0.append(r0_)
            t1.append(r1_)
        return t1, t0
    
    def decompose(self, w, alpha):
        w0, w1 = [], []
        for i in range(self.k):
            m0, m1 = [], []
            for c in w[i]:
                #def decompose(r, alpha, q):
                r_plus = c % self.q
                r0 = Integer(r_plus) % (2 * alpha)
                
                if r_plus - r0 == self.q - 1:
                    r1 = 0
                    r0 -= 1
                else:
                    r1 = (r_plus - r0) // (2 * alpha)
                m0.append(r0), m1.append(r1)
            w0.append(m0), w1.append(m1)
        return w1, w0
    
    def check_norm_bound(self, matrix, bound):
        for row in matrix:
            for p in row:
                if p >= bound:
                    print(p, bound)
                    return True
        return False
        
DiliThium2 = Dilithium("dilithium2")
DiliThium3 = Dilithium("dilithium3")
DiliThium5 = Dilithium("dilithium5")

In [3]:
# Example of signing
pk, sk = Dilithium2.keygen()
print(f"Public-Key: {pk}\nlen: {len(pk)}")
print(f"Secret-Key: {sk}\nlen: {len(sk)}")
msg = b"Your message signed by Dilithium"
sig = Dilithium2.sign(sk, msg)
print(f"Sign: {sig}\nlen: {len(sig)}")
print(Dilithium2.verify(pk, msg, sig))

Public-Key: b'\xb3{ze\x89b\xe2\xc2\x8b\xb1\x1ab\x9a\xdc\xf0\xaa\x9b\xb4&2\xc5\xbdW\xe9\x8a-Q-%U^\xa8\x80\x89\xcbs+\xa2\xf2\xa0\x85\x89o\x8f\x0c\x95\x82\xf4V\xd0\xcd\x15\x18T\x82H2\x93V\'\x02\x165\x92\xb0\xca\xb2\x10I}D\xff\xf3\x88(C\xb5\x867=\xf5\x92\xfe\xc3\x87\x00\x81\x82\x13\xfb\xb3\xbb\xd3uGR\xcb\xa6`0\xde\x8e7\xa6"B\x1d\xf6\xcf\xde\\\xc9\xb0\x8d\xc2\xd1\xef\r>\xe9\xfa\xf6\xe7\xaa EJ\x14\x9e\x8b,\xc36\x90\xad\r\x8f\xe5\xe5j\x0e|\xabn6.\xc7C\xd5\x15\xdb\xb1h\xd6\x9f\x99\x81+\xe3\xff"\xbe\xcd\xf4\x9eKeS)\xa9\xcd\xb7\xdc\xe9Ebc\x04\xcb\xe9\xe7\x93\xb2WB(\x84\x1a?|Q^UT\xec\x12\x97\xab~\x95\xb2\x84!\xd6\xbd\x11\xb3\xde\xfd\x9f\x9cq\xc5\'\xc2\xb2\xf7Q\x9d\xace\xe8\x02\x95\xceG\xbc\xec\x07\x7f\xf8\xfc\xb2~\xe7^\xc9^\x1f\xa5\xcfX\x9dn\xcb\xeb\xbc\xfa\xd6k\xe1d{\x02\x12#\xd6A\xe1\x1eN"\xd5\xc1\xd1\x1ca(\xdah\xc2\x1e\xf8{B\xe1D; \xc0\x80\xc2\x90\xad\xbc\x98\x9b\x13\xbc#\x8fQU\xfe\r\xf5I\xae\x1b\x02A\x98\x0c\xa0\xfa\xfd\x1fF\xe8#@\xeb\x82\xb2\xb57P\xec\xc0\xe9\x86\xaeE\x01\xb5\xcbo(\x80\xc5+\

In [4]:
pk, sk = Dilithium3.keygen()
print(f"Public-Key: {pk}\nlen: {len(pk)}")
print(f"Secret-Key: {sk}\nlen: {len(sk)}")
msg = b"Your message signed by Dilithium"
sig = Dilithium3.sign(sk, msg)
print(f"Sign: {sig}\nlen: {len(sig)}")
print(Dilithium3.verify(pk, msg, sig))

Public-Key: b'\x8c\xb3"\x90G\xcee\xe7\xf6\xbcb\xe5p\xea\\\x047\x81 \xec0\xc1\x18\xc4\xde\xc1\n\xe57\xdf\xf3\xa2aa;H\x1e\x17u\xa4\x8f\xb51\x80\x17)\xbe\xb1\xef\x1cY\xfbku3\xb7\x85\x7fQ\x88}\xcbo\x9a\x8aZy2\x97\xd4\xb6\x07\x17\xd0\\\xf0\xea\xec.)\x87\xfb\x12K-\x99\x9a\x91\xf3[+\xc7$\xce\xd8\xf0B\xfa\x86\xfe\x1a\xa1\xb5p\x82\xf1M3\xdf \x16\xb6\xfc2\x12\xbe&\xc4\x06\x86\xcd\xe7}\xdd\xaa\x14X&\xe8\xb6U\xff\x12\x1a\x9c\xb0\xf7\xfc\x15\xa0\x0b\x9a\x0f\xa1-\xc0\x15\x7f:(\x11q\xce\xba\xbf;\xc3K\xfe\x84\xbb\xea\x11\xee*\x87\xc8\xa3S\xad\xea\x16-w\xd4Z\x12\xf3\xab\xd7\xb4\x9a\x9d1\x85\x1b\xbd\x89\xb2B\x83H5\x8e\x0c\x9f1\xfcyI\xb1\x08]\xb1\x8fr7U\x94\xbeJ\x87\x08\xc8\xdb"@9v\xd4\xd6`\x9fL\xc5\xd9\xb0\x88\x80\xbaU\x87\n\xc5\xa8\xf5Y\xaa\x0e\x04\x92Fh\x93\x04\xde\x96\x12r\x10\nXD4>1sG \x99lWt\xaa\xd2k\xafoe\x14.\\q\xf5l\x00\xca\xef\x1d9;i[\x93\xcc!]\xd9b[\xe2\x0c\xfa\xe8\xfa\xcdp\xd0\xd9M\x12\xb8&\xc8\xe4\xbc\xa5\xc6\x18y+\xd5\xd6\xac!\xf2\xd3\x82\xa6jE@\xa5\x17\xcaW\xb1\x1c,\xbf\xceDc\xcf3\x16\xc9\

In [5]:
pk, sk = Dilithium5.keygen()
print(f"Public-Key: {pk}\nlen: {len(pk)}")
print(f"Secret-Key: {sk}\nlen: {len(sk)}")
msg = b"Your message signed by Dilithium"
sig = Dilithium5.sign(sk, msg)
print(f"Sign: {sig}\nlen: {len(sig)}")
print(Dilithium5.verify(pk, msg, sig))

Public-Key: b'o[}A\xf6\x04\xd6\xe6\xe8\xa0(5\x00\xf2\x86p\x064\tw%E;C\xb7\xbbS\xc4\xd6\xc9j\xe1\x90m\x1aN\'X-\xee\x1d,\xee\xf8\xda\x87\xb2\xe5y\xf0\xcc\xa7\xa9\xc1P}\xec*I\x89\x87\xf5\xf9\xde\xff\xf0\xb4\x0c\x12\x01\x9dk\xda\xb7\n\xeb\xb2\x0f\xed\x82\\+\xa5D\xae\x8bR\xaf #\xd0\xb2\x90T`+\x15\xd9\x8c\xb4\x98]\xa2;\xe9\x85\xa2\xe0\x0fB\x08\xf9\xe7B\xf7\xa9a!\x1a\xf0\x83`\x12\xd2}\xf8\xf32.}8\xeefSJX\x0b3\xd5\xeb\xf0\x06b\x7f\x8c3\xd7\x8d\x176J\x17\xb9\x0f3 \x01\x9a(\xe2iQP\xc6\xd8h\xfd\xb0\x96\x0f)\x0b\xdcug\x8c\x1e\x12\n%\xf0\';T\'\xc0\xac/\x9bE\xc2m_1\x89\xebT\xdfgj\x91w\xf2[\xf6\x8f\x93\xcc\xf63\x13\xcfF\x03\xe7iM?\x9a\x11\xd2_\xfd{aD\x16jO\x91\xa7\xdfD\xb2#@\x92\xe0$Q\xdb\x0b\x1e\xe9U\xd1tcq\x11i8\x18P\x85\xb1\')JD\xf1\x92\x05"\x07O\xac\x92o\x98\x00\xe2\x94\x9eg\x1db|\xcb,\xd8\xdd\x93\xc8\xdb\xe4\x14[\xa1\xad\x99r\xf8\xf5\xf8r^\xf1\xb0\xd4\xd2\x8a\xa2\x83\xf4\x9dS\\9)6\xdb\x84\x17\xfb8\x9c\x98\xdc\xc4\xb7\x8c\xa4x\xc7\x1ca\xf7\xb8-\xbd\x15\x11\x8c\x14\xd9\xd4P/\x93Llf\x0c\x8b/m\xf5(\